## Getting Foot Traffic Data From Dewey

In [ ]:
#required libraries
# !pip install google-cloud-storage
# !pip install python-dotenv
# !pip install fsspec
# !pip install gcsfs
# !pip install google-cloud-bigquery
# !pip install pandas-gbq

## Setting Connection

In [1]:
from dotenv import load_dotenv
import os
from google.cloud import storage

load_dotenv()

client = storage.Client()
bucket = client.bucket(os.getenv('GOOGLE_CLOUD_BUCKET'))

files within the bucket are stored as objects called "blobs". We can check the list of all the blobs in our bucket as follows.

In [2]:
blobs = bucket.list_blobs()

for blob in blobs:
    print(blob)

<Blob: businesstraffic, advantrafic/2019-01-01/data_01ba9160-0004-a591-0042-fa0701b2b1e2_53_1_0.csv.gz, 1740252849243817>
<Blob: businesstraffic, advantrafic/2019-01-01/data_01ba9160-0004-a591-0042-fa0701b2b1e2_53_1_1.csv.gz, 1740252879354773>
<Blob: businesstraffic, advantrafic/2019-01-01/data_01ba9160-0004-a591-0042-fa0701b2b1e2_53_1_10.csv.gz, 1740254075828346>
<Blob: businesstraffic, advantrafic/2019-01-01/data_01ba9160-0004-a591-0042-fa0701b2b1e2_53_1_11.csv.gz, 1740254165375700>
<Blob: businesstraffic, advantrafic/2019-01-01/data_01ba9160-0004-a591-0042-fa0701b2b1e2_53_1_12.csv.gz, 1740254252737748>
<Blob: businesstraffic, advantrafic/2019-01-01/data_01ba9160-0004-a591-0042-fa0701b2b1e2_53_1_13.csv.gz, 1740254456805548>
<Blob: businesstraffic, advantrafic/2019-01-01/data_01ba9160-0004-a591-0042-fa0701b2b1e2_53_1_14.csv.gz, 1740254487986647>
<Blob: businesstraffic, advantrafic/2019-01-01/data_01ba9160-0004-a591-0042-fa0701b2b1e2_53_1_15.csv.gz, 1740254513421250>
<Blob: businesstra

Downloading two monthly data in 2019 and 2024.

In [3]:
import pandas as pd
# Initialize empty list to store Philadelphia dataframes
phillyfile = []

# Iterate through blobs
blobs = bucket.list_blobs(prefix='advantrafic/2019-01-01')
for blob in blobs:
    try:
        # Create blob path
        blobpath = f'gs://{bucket.name}/{blob.name}'  # Added missing forward slash
        
        # Read the CSV
        blobdata = pd.read_csv(blobpath, compression='gzip')
        
        # Filter for Philadelphia
        blobphilly = blobdata[blobdata['CITY']=='Philadelphia']
        
        # Append to list of dataframes
        phillyfile.append(blobphilly)
    except Exception as e:
        print(f"Error processing {blob.name}: {e}")

# Concatenate all Philadelphia data outside the loop
if phillyfile:  # Check if list is not empty
    final_philly_df = pd.concat(phillyfile, ignore_index=True)
else:
    print("No Philadelphia data found")

/var/folders/mm/m098rbs1529c7x072qh31ncw0000gn/T/ipykernel_3406/1509496212.py:13: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  blobdata = pd.read_csv(blobpath, compression='gzip')
/var/folders/mm/m098rbs1529c7x072qh31ncw0000gn/T/ipykernel_3406/1509496212.py:13: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  blobdata = pd.read_csv(blobpath, compression='gzip')
/var/folders/mm/m098rbs1529c7x072qh31ncw0000gn/T/ipykernel_3406/1509496212.py:13: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  blobdata = pd.read_csv(blobpath, compression='gzip')
/var/folders/mm/m098rbs1529c7x072qh31ncw0000gn/T/ipykernel_3406/1509496212.py:13: DtypeWarning: Columns (18,34,36,37,38,39,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  blobdata = pd.read_csv(blobpath, compression='gzip')
/var/folders/mm

In [4]:
final_philly_df.head()

,PLACEKEY,PARENT_PLACEKEY,SAFEGRAPH_BRAND_IDS,LOCATION_NAME,BRANDS,STORE_ID,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,LATITUDE,...,RELATED_SAME_DAY_BRAND,RELATED_SAME_MONTH_BRAND,POPULARITY_BY_HOUR,POPULARITY_BY_DAY,DEVICE_TYPE,NORMALIZED_VISITS_BY_STATE_SCALING,NORMALIZED_VISITS_BY_REGION_NAICS_VISITS,NORMALIZED_VISITS_BY_REGION_NAICS_VISITORS,NORMALIZED_VISITS_BY_TOTAL_VISITS,NORMALIZED_VISITS_BY_TOTAL_VISITORS
0,22k-222@628-pj5-swk,NaN,NaN,John's Pizza Express,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511,40.012031,...,"{""Western Union"":33,""Sunoco"":21,""Dunkin'"":14,""...","{""Western Union"":108,""Sunoco"":82,""Dunkin'"":75,...","[3,3,6,4,2,1,7,9,20,20,21,11,23,19,18,10,13,15...","{""Monday"":18,""Tuesday"":18,""Wednesday"":29,""Thur...","{""android"":67,""ios"":60}",2737.0,0.000408,0.001453,0.000004,0.000321
1,22d-222@628-pmf-qmk,NaN,NaN,Yeeroh,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511,39.943614,...,"{""Western Union"":14,""Starbucks"":12,""CVS"":11,""C...","{""Western Union"":43,""Starbucks"":39,""Dunkin'"":3...","[0,2,0,0,1,0,1,1,2,2,4,4,4,9,3,4,7,7,9,9,7,5,5,1]","{""Monday"":6,""Tuesday"":9,""Wednesday"":10,""Thursd...","{""android"":31,""ios"":16}",1053.0,0.000153,0.000548,0.000002,0.000124
2,223-223@628-pm3-wc5,NaN,NaN,Soul & Busan,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511,39.933357,...,"{""Western Union"":68,""Keyme Kiosk"":37,""Dunkin'""...","{""Western Union"":158,""Dunkin'"":129,""Starbucks""...","[18,19,21,19,20,20,22,32,23,23,27,28,23,23,23,...","{""Monday"":33,""Tuesday"":53,""Wednesday"":49,""Thur...","{""ios"":86,""android"":83}",4229.0,0.000637,0.002295,0.000006,0.000496
3,232-223@628-pgh-xkf,NaN,NaN,Eli Chinese Food,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511,39.987824,...,"{""Western Union"":14,""Sunoco"":7,""Ivy Rehab"":7,""...","{""Western Union"":52,""Sunoco"":40,""Dunkin'"":40,""...","[1,0,0,1,1,1,3,7,5,5,3,1,3,2,10,7,4,3,3,2,1,2,...","{""Monday"":10,""Tuesday"":5,""Wednesday"":10,""Thurs...","{""android"":39,""ios"":24}",1050.0,0.000153,0.000544,0.000001,0.000123
4,zzy-22q@628-pmb-835,NaN,NaN,Pearl's Oyster Bar,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511,39.953210,...,"{""Western Union"":9889,""Dunkin'"":9886,""Hilton G...","{""Western Union"":23681,""Dunkin'"":20850,""Starbu...","[1041,877,776,656,668,1043,2243,5293,6295,6295...","{""Monday"":5905,""Tuesday"":8566,""Wednesday"":8605...","{""android"":13560,""ios"":13430}",812257.0,0.119926,0.430869,0.001167,0.095219


In [25]:
final_philly_df.info

<bound method DataFrame.info of                   PLACEKEY PARENT_PLACEKEY SAFEGRAPH_BRAND_IDS  \
0      22k-222@628-pj5-swk             NaN                 NaN   
1      22d-222@628-pmf-qmk             NaN                 NaN   
2      223-223@628-pm3-wc5             NaN                 NaN   
3      232-223@628-pgh-xkf             NaN                 NaN   
4      zzy-22q@628-pmb-835             NaN                 NaN   
...                    ...             ...                 ...   
56299  233-223@628-pgf-ghq             NaN                 NaN   
56300  zzy-224@628-ph4-ghq             NaN                 NaN   
56301  23s-222@628-pfz-bzf             NaN                 NaN   
56302  zzw-222@628-p8t-4vz             NaN                 NaN   
56303  225-222@628-pg9-9cq             NaN                 NaN   

                      LOCATION_NAME BRANDS  STORE_ID  \
0              John's Pizza Express    NaN       NaN   
1                            Yeeroh    NaN       NaN   
2      

In [6]:
from shapely.geometry import Point
import geopandas as gpd

geometry = [Point(xy) for xy in zip(final_philly_df['LONGITUDE'], final_philly_df['LATITUDE'])]
    
gfinal_philly_df = gpd.GeoDataFrame(final_philly_df, geometry=geometry)
    
gfinal_philly_df.crs = "EPSG:4326" 

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import json
import folium
from folium.plugins import HeatMap
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import panel as pn
import warnings

# Ignore FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Initialize extensions 
pn.extension()
hv.extension('bokeh')


# Create a list to store the hourly data
hourly_data = []

# Iterate through each row in the dataframe
for idx, row in gfinal_philly_df.iterrows():
    # Get the category and location name
    category = row['SUB_CATEGORY']
    location_name = row['LOCATION_NAME']
    
    # Check if POPULARITY_BY_HOUR exists and process it
    if 'POPULARITY_BY_HOUR' in gfinal_philly_df.columns:
        hourly_values = row['POPULARITY_BY_HOUR']
        
        # Parse the hourly values if they're in string format
        if isinstance(hourly_values, str):
            try:
                # Try parsing as JSON first
                hourly_values = json.loads(hourly_values.replace("'", '"'))
            except:
                # Try parsing as Python list
                try:
                    hourly_values = ast.literal_eval(hourly_values)
                except:
                    # If parsing fails, skip this row
                    continue
        
        # Handle list format
        if isinstance(hourly_values, list):
            for hour, count in enumerate(hourly_values):
                hourly_data.append({
                    'SUB_CATEGORY': category,
                    'LOCATION_NAME': location_name,
                    'HOUR': hour,
                    'VISITS': count
                })
        
        # Handle dictionary format
        elif isinstance(hourly_values, dict):
            for hour_str, count in hourly_values.items():
                try:
                    hour = int(hour_str)
                    hourly_data.append({
                        'SUB_CATEGORY': category,
                        'LOCATION_NAME': location_name,
                        'HOUR': hour,
                        'VISITS': count
                    })
                except:
                    # Skip if hour can't be converted to int
                    continue

# Create a dataframe from the hourly data
hourly_df = pd.DataFrame(hourly_data)
print(f"Created hourly dataframe with {len(hourly_df)} rows")

# Step 2: Aggregate hourly data by category
hourly_by_category = hourly_df.groupby(['SUB_CATEGORY', 'HOUR'])['VISITS'].mean().reset_index()
print(f"Aggregated hourly data by {hourly_by_category['SUB_CATEGORY'].nunique()} categories")

# Step 3: Create a heatmap using folium
print("Creating geospatial heatmap...")

# Get the normalized visit column
if 'NORMALIZED_VISITS_BY_TOTAL_VISITS' in gfinal_philly_df.columns:
    norm_col = 'NORMALIZED_VISITS_BY_TOTAL_VISITS'
elif 'NORMALIZED_VISITS_BY_TOTAL_VISITORS' in gfinal_philly_df.columns:
    norm_col = 'NORMALIZED_VISITS_BY_TOTAL_VISITORS'
else:
    print("No normalized visits column found, using placeholder")
    gfinal_philly_df['normalized_visits'] = 1
    norm_col = 'normalized_visits'

# Get center coordinates for the map
center_lat = gfinal_philly_df['LATITUDE'].mean()
center_lon = gfinal_philly_df['LONGITUDE'].mean()

# Create a folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Prepare heatmap data: [[lat, lon, intensity], ...]
heat_data = [[row['LATITUDE'], row['LONGITUDE'], row[norm_col] * 1000000]  # Scale for visibility
             for idx, row in gfinal_philly_df.iterrows() if not pd.isna(row[norm_col])]

# Add heatmap layer
HeatMap(heat_data, radius=15, blur=10, max_zoom=13).add_to(m)

# Save the heatmap
heatmap_file = 'restaurant_heatmap.html'
m.save(heatmap_file)
print(f"Saved heatmap to {heatmap_file}")

# Step 4: Create hourly popularity chart by category using hvplot
print("Creating hourly popularity chart...")

# Get top categories by total visits
top_n = 10  # Number of top categories to display
top_categories = hourly_by_category.groupby('SUB_CATEGORY')['VISITS'].sum().nlargest(top_n).index.tolist()

# Filter for top categories
top_hourly_data = hourly_by_category[hourly_by_category['SUB_CATEGORY'].isin(top_categories)]

# Create the hvplot
hourly_plot = top_hourly_data.hvplot.line(
    x='HOUR',
    y='VISITS',
    by='SUB_CATEGORY',
    title='Hourly Popularity of Businesses',
    xlabel='Hour of Day',
    ylabel='Average Visit Count',
    grid=True,
    legend='right',
    width=800,
    height=500
)


Created hourly dataframe with 904848 rows
Aggregated hourly data by 275 categories
Creating geospatial heatmap...
Saved heatmap to restaurant_heatmap.html
Creating hourly popularity chart...


In [18]:
hourly_plot

:NdOverlay   [SUB_CATEGORY]
   :Curve   [HOUR]   (VISITS)

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
import hvplot.pandas
import holoviews as hv
from holoviews import opts, dim
import param
import colorcet as cc
import folium
from folium.plugins import HeatMap, MarkerCluster
import matplotlib.cm as cm
import branca.colormap as bcm
import json
import ast
import warnings
from io import StringIO

# Ignore warnings
warnings.filterwarnings('ignore')


# Step 1: Extract normalized visits information for color coding
# Get the normalized visit column
if 'NORMALIZED_VISITS_BY_TOTAL_VISITS' in gfinal_philly_df.columns:
    norm_col = 'NORMALIZED_VISITS_BY_TOTAL_VISITS'
elif 'NORMALIZED_VISITS_BY_TOTAL_VISITORS' in gfinal_philly_df.columns:
    norm_col = 'NORMALIZED_VISITS_BY_TOTAL_VISITORS'
else:
    print("No normalized visits column found, using placeholder")
    gfinal_philly_df['normalized_visits'] = 1
    norm_col = 'normalized_visits'


# Create a list to store the hourly data
hourly_data = []

# Iterate through each row in the dataframe
for idx, row in gfinal_philly_df.iterrows():
    # Get the category and location name
    category = row['SUB_CATEGORY']
    location_name = row['LOCATION_NAME']
    
    # Check if POPULARITY_BY_HOUR exists and process it
    if 'POPULARITY_BY_HOUR' in gfinal_philly_df.columns:
        hourly_values = row['POPULARITY_BY_HOUR']
        
        # Parse the hourly values if they're in string format
        if isinstance(hourly_values, str):
            try:
                # Try parsing as JSON first
                hourly_values = json.loads(hourly_values.replace("'", '"'))
            except:
                # Try parsing as Python list
                try:
                    hourly_values = ast.literal_eval(hourly_values)
                except:
                    # If parsing fails, skip this row
                    continue
        
        # Handle list format
        if isinstance(hourly_values, list):
            for hour, count in enumerate(hourly_values):
                hourly_data.append({
                    'SUB_CATEGORY': category,
                    'LOCATION_NAME': location_name,
                    'HOUR': hour,
                    'VISITS': count
                })
        
        # Handle dictionary format
        elif isinstance(hourly_values, dict):
            for hour_str, count in hourly_values.items():
                try:
                    hour = int(hour_str)
                    hourly_data.append({
                        'SUB_CATEGORY': category,
                        'LOCATION_NAME': location_name,
                        'HOUR': hour,
                        'VISITS': count
                    })
                except:
                    # Skip if hour can't be converted to int
                    continue

# Create a dataframe from the hourly data
hourly_df = pd.DataFrame(hourly_data)

# Aggregate hourly data by category
hourly_by_category = hourly_df.groupby(['SUB_CATEGORY', 'HOUR'])['VISITS'].mean().reset_index()

# Get top categories by total visits 
top_n = 10  # Number of top categories to display
top_categories = hourly_by_category.groupby('SUB_CATEGORY')['VISITS'].sum().nlargest(top_n).index.tolist()

# Filter for top categories
top_hourly_data = hourly_by_category[hourly_by_category['SUB_CATEGORY'].isin(top_categories)]

# Step 3: Create interactive map with Panel/HoloViews
print("Creating interactive map...")

# Create a point plot with hover information
geo_plot = gfinal_philly_df.hvplot.points(
    'LONGITUDE', 'LATITUDE',
    geo=True,
    color=norm_col,  # Use the variable, not the string 'norm_col'
    size=norm_col,   # Use the variable, not the string 'norm_col'
    tiles='CartoDark',
    frame_height=500,
    frame_width=700,
    title='Restaurant Distribution by Visit Rate',
    hover_cols=['LOCATION_NAME', 'TOP_CATEGORY', 'SUB_CATEGORY', norm_col],
    cmap='viridis',
    alpha=0.7,
    legend=False,
    clabel='Normalized Visit Rate',
    # Add these parameters to ensure zoom works
    tools=['pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'],
    active_tools=['wheel_zoom']  # Enable wheel zoom by default
)

# Step 4: Create interactive hourly chart
print("Creating interactive hourly chart...")

hourly_plot = top_hourly_data.hvplot.line(
    x='HOUR',
    y='VISITS',
    by='SUB_CATEGORY',
    line_width=3,
    title='Hourly Popularity by Restaurant Category',
    xlabel='Hour of Day',
    ylabel='Average Visit Count',
    grid=True,
    frame_height=400,
    frame_width=700,
    legend='right'
)


# Create a class for interactive dashboard
class RestaurantDashboard(param.Parameterized):
    category = param.ListSelector(default=[], objects=all_categories)
    hour_of_day = param.Integer(default=12, bounds=(0, 23))
    
    def get_filtered_map(self):
        # Filter based on selected categories
        if not self.category:
            filtered_df = gfinal_philly_df
        else:
            filtered_df = gfinal_philly_df[gfinal_philly_df['SUB_CATEGORY'].isin(self.category)]
        
        # Create the plot
        plot = filtered_df.hvplot.points(
            'LONGITUDE', 'LATITUDE',
            geo=True,
            color='scaled_visits',
            size='scaled_visits',
            tiles='CartoDark',
            frame_height=500,
            frame_width=700,
            title=f'Restaurant Distribution by Visit Rate',
            hover_cols=['LOCATION_NAME', 'SUB_CATEGORY', 'TOP_CATEGORY', norm_col],
            cmap='viridis',
            alpha=0.7,
            legend=False,
            clabel='Normalized Visit Rate'
        )
        return plot
    
    def get_hourly_chart(self):
        # Filter based on selected categories
        if not self.category:
            filtered_df = top_hourly_data
        else:
            filtered_df = hourly_by_category[hourly_by_category['SUB_CATEGORY'].isin(self.category)]
            # If filtered data is empty, use all top categories
            if len(filtered_df) == 0:
                filtered_df = top_hourly_data
        
        # Create the plot
        plot = filtered_df.hvplot.line(
            x='HOUR',
            y='VISITS',
            by='SUB_CATEGORY',
            line_width=3,
            title=f'Hourly Popularity by Restaurant Category',
            xlabel='Hour of Day',
            ylabel='Average Visit Count',
            grid=True,
            frame_height=400,
            frame_width=700,
            legend='right'
        )
        
        # Add a vertical line for current hour
        vline = hv.VLine(self.hour_of_day).opts(
            color='red', 
            line_width=2,
            line_dash='dashed'
        )
        
        return plot * vline
    
    def view(self):
        # Create the dashboard layout
        return pn.Column(
            pn.pane.Markdown('# Philadelphia Business Analytics Dashboard'),
            
            pn.Row(
                pn.Column(
                    pn.pane.Markdown('## Filters'),
                    pn.Param(
                        self.param,
                        widgets={
                            'category': pn.widgets.MultiSelect(size=6),
                            'hour_of_day': pn.widgets.IntSlider
                        }
                    ),
                    width=300
                ),
                pn.Column(
                    pn.pane.Markdown('## Geospatial Distribution'),
                    pn.bind(self.get_filtered_map, watch=True),
                    pn.pane.Markdown('## Hourly Popularity by Category'),
                    pn.bind(self.get_hourly_chart, watch=True)
                )
            )
        )

# Create the dashboard
dashboard = RestaurantDashboard()

# Save the dashboard
output_file = 'philadelphia_dashboard.html'
dashboard.view().save(output_file)

Creating interactive map...
Creating interactive hourly chart...


In [24]:
dashboard.view().servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'609dd2a0-f698-4173-aef8-06a9a06f8de1': {'version…